In [49]:
import pandas as pd
import numpy as np
from tabulate import tabulate

<i>Question 7</i>

Find the balance of tourists per year per country and continent. Balance is the difference of number of incoming versus outgoing tourists<br>

In [50]:
## to add comma after every thousandth place in number to avoid scientific notation conversion by tabular
def format_number(num):
    return f"{num:,}"

In [51]:
years = ['2014','2016','2017','2019','2020','2021','2022']
regions = ['North America', 'Central & South America', 'Western Europe', 'Eastern Europe', 'Africa', 'West Asia', 'South Asia', 'South East Asia', 'East Asia', 'Australasia']

# Nationality wise outgoing tourist data not available for 2013 and 2018.
# Also incoming tourist data and outgoing tourists data for 2014 is exactly same
# which is not possible in real world data.
# Similar is the case with data from 2017 where the balance is coming out to be 
# zero for almost all the countries except few.

In [52]:
data_in = {}
data_out = {}

cube_in = {}
cube_out = {}

for y in years:
    data_in[f'{y}'] = pd.read_excel(rf"data\TourismData-{y}\NATIONALITY-WISE GENDER-WISE DISTRIBUTION OF FTAs IN INDIA.xlsx")
    # data_in[f'{y}'].set_index('Country', inplace = True)
    # data_in[f'{y}'].index.name = None
    data_in[f'{y}'] = data_in[f'{y}'][['Country','Total']]
    data_out[f'{y}'] = pd.read_excel(rf"data\TourismData-{y}\QUARTERWISE PERCENTAGE DISTRIBUTION OF DESTINATION COUNTRY WISE DEPARTURES OF INDIAN NATIONALS.xlsx")
    # data_out[f'{y}'].set_index('Country of Nationality', inplace = True)
    # data_out[f'{y}'].index.name = None
    data_out[f'{y}'] = data_out[f'{y}'][['Country of Nationality','Arrivals (in numbers)']]

for y in years:
    df_in = data_in[f'{y}']
    df_out = data_out[f'{y}']

    r_in = df_in[df_in['Country'].isin(regions)].index.tolist()
    r_out = df_out[df_out['Country of Nationality'].isin(regions)].index.tolist()
    split_data_in = []
    split_data_out = []

    for i in range(len(r_in)):
        start_idx = r_in[i]
        end_idx = r_in[i+1] if i+1 < len(r_in) else len(df_in)
        section = df_in.iloc[start_idx:end_idx].values.tolist()
        split_data_in.append(section)
    
    for i in range(len(r_out)):
        start_idx = r_out[i]
        end_idx = r_out[i+1] if i+1 < len(r_out) else len(df_out)
        section = df_out.iloc[start_idx:end_idx].values.tolist()
        split_data_out.append(section)
    
    for i in range(len(split_data_in)):
        chunk = split_data_in[i]
        total = 0
        for j in range(1, len(chunk)):
            total += chunk[j][1]
        chunk[0][1] = int(total)
    
    for i in range(len(split_data_out)):
        chunk = split_data_out[i]
        total = 0
        for j in range(1, len(chunk)):
            total += chunk[j][1]
        chunk[0][1] = int(total)
    
    cube_in[f'{y}'] = split_data_in
    cube_out[f'{y}'] = split_data_out   





In [53]:
ans_in = {}
ans_out = {}

for y in years:
    cube1 = cube_in[f'{y}']
    cube2 = cube_out[f'{y}']
    ans1 = {}
    ans2 = {}
    ans1['countries'] = []
    ans1['regions'] = []
    ans2['countries'] = []
    ans2['regions'] = []

    for chunk in cube1:
        ans1['regions'].append(chunk[0])
        for c_data in chunk[1:]:
            if(c_data[0] != 'Others'):
                ans1['countries'].append(c_data)
    
    for chunk in cube2:
        ans2['regions'].append(chunk[0])
        for c_data in chunk[1:]:
            if(c_data[0] != 'Others'):
                ans2['countries'].append(c_data)

    ans_in[f'{y}'] = ans1
    ans_out[f'{y}'] = ans2

In [54]:
def find_data(dict_name, year, type, name): # year and type to be provided in string format. type is 'countries' or 'regions'.
    for row in dict_name[year][type]:
        if row[0].lower() == name.lower():
            retval = row.copy()
            retval[1] = int(retval[1])
            return retval[1]
    return None

In [55]:
countries_in = set()
countries_out = set()
for y in years:
    data = ans_in[f'{y}']['countries']
    for country in [c_data[0] for c_data in data]:
        countries_in.add(country)
for y in years:
    data = ans_out[f'{y}']['countries']
    for country in [c_data[0] for c_data in data]:
        countries_out.add(country)
    
countries = sorted(list(countries_out.intersection(countries_in)))

In [56]:
table_r = {}
table_c = {}
table_r['Continent'] = regions
table_c['Country'] = countries

for y in years:
    table_c[f'{y}'] = []
    for c in countries:
        incoming = find_data(ans_in, f'{y}', 'countries', c)
        outgoing = find_data(ans_out, f'{y}', 'countries', c)
        table_c[f'{y}'].append(incoming-outgoing) if (incoming!=None and outgoing!=None) else table_c[f'{y}'].append(None)

    table_r[f'{y}'] = []
    for r in regions:
        incoming = find_data(ans_in, f'{y}', 'regions', r)
        outgoing = find_data(ans_out, f'{y}', 'regions', r)
        table_r[f'{y}'].append(format_number(incoming-outgoing))



In [57]:
print(tabulate(pd.DataFrame(table_c), headers='keys', tablefmt='grid'))
print("Table 1: Balance of tourists per year per country")

+----+----------------+--------+---------+---------+-------------------+-------------------+-------------------+-------------------+
|    | Country        |   2014 |    2016 |    2017 |              2019 |              2020 |              2021 |              2022 |
+====+================+========+=========+=========+===================+===================+===================+===================+
|  0 | Afghanistan    |      0 |  -25846 |       0 |  109726           |   41888           |   32433           |     420           |
+----+----------------+--------+---------+---------+-------------------+-------------------+-------------------+-------------------+
|  1 | Argentina      |      0 |   -2567 |       0 |   10004           |    3596           |      97           |    3365           |
+----+----------------+--------+---------+---------+-------------------+-------------------+-------------------+-------------------+
|  2 | Australia      |      0 |  -30618 |       0 | -222298         

In [58]:
print(tabulate(pd.DataFrame(table_r), headers='keys', tablefmt='grid'))
print("Table 2: Balance of tourists per year per continent")

+----+-------------------------+--------+-----------+-----------+-------------+------------+------------+-------------+
|    | Continent               |   2014 | 2016      | 2017      | 2019        | 2020       | 2021       | 2022        |
+====+=========================+========+===========+===========+=============+============+============+=============+
|  0 | North America           |      0 | -98,180   | 0         | -708,925    | -303,209   | -445,322   | -814,417    |
+----+-------------------------+--------+-----------+-----------+-------------+------------+------------+-------------+
|  1 | Central & South America |      0 | -13,337   | 0         | 28,893      | 5,159      | -41,687    | -25,140     |
+----+-------------------------+--------+-----------+-----------+-------------+------------+------------+-------------+
|  2 | Western Europe          |      0 | -104,261  | 0         | -111,311    | 13,153     | -442,642   | -758,563    |
+----+-------------------------+--------

<i>Question 8</i>

Using the answer in Q7, find the top-5 countries with highest positive and highest
negative balances per year.<br>

In [60]:
df = pd.DataFrame(table_c)

df.loc[:, df.columns[1:]] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

positive_top5 = {}
for year in df.columns[1:]:  # Skip the 'Country' column
    positive_top5[year] = df.nlargest(5, year, 'all')[['Country', year]]
    
negative_top5 = {}
for year in df.columns[1:]:  # Skip the 'Country' column
    negative_top5[year] = df.nsmallest(5, year, 'all')[['Country', year]] 
    
print("Top 5 positive balances for each year:")
for year, top5 in positive_top5.items():
    if year == '2014' or year == '2017':
        continue
    print(f"{year}:\n{top5}\n")

print("Top 5 negative balances for each year:")
for year, top5 in negative_top5.items():
    if year == '2014' or year == '2017':
        continue
    print(f"{year}:\n{top5}\n")

# skipping years 2014 and 2017 because all the balances were zero resulting in no top-5 or bottom-5 balances

Top 5 positive balances for each year:
2016:
         Country     2016
5     Bangladesh  1164752
40      Pakistan    60454
30      Maldives    51269
33       Myanmar    40883
46  Saudi Arabia     8629

2019:
         Country       2019
5     Bangladesh  2077239.0
58            UK   174727.0
45  Russian Fed.   151252.0
0    Afghanistan   109726.0
26         Japan   102712.0

2020:
         Country      2020
5     Bangladesh  397211.0
45  Russian Fed.   90960.0
0    Afghanistan   41888.0
16        France   41288.0
58            UK   32392.0

2021:
        Country      2021
5    Bangladesh  143835.0
0   Afghanistan   32433.0
43     Portugal   22105.0
22         Iraq    7252.0
7        Bhutan    3792.0

2022:
          Country      2022
5      Bangladesh  805066.0
45   Russian Fed.   55223.0
43       Portugal   46774.0
50      Sri Lanka   37298.0
44  Rep. of Korea   36970.0

Top 5 negative balances for each year:
2016:
          Country   2016
59            USA -79980
45   Russian Fed. -51